In [2]:
import os
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split

In [16]:
# 路径配置
data_dir = os.path.join('data', 'train')
label_file = os.path.join('data', 'train_labels.csv')

# 读取标签数据
labels = pd.read_csv(label_file)

# 初始化数据列表
images = []
image_labels = []

# 遍历每个文件夹和图片
for subdir in os.listdir(data_dir):
    subdir_path = os.path.join(data_dir, subdir)
    print('Processing', subdir_path)
    if os.path.isdir(subdir_path):
        for file in os.listdir(subdir_path):
            if file.endswith('.JPG'):
                file_path = os.path.join(subdir_path, file)
                # 读取图像
                image = Image.open(file_path)
                # 调整图像大小
                image = image.resize((256, 256))
                # 添加到数据列表
                images.append(
                    {
                        'image': image,
                        'file_name': file,
                    }
                )


Processing data\train\hand
Processing data\train\hand_outliers
Processing data\train\neutral_bg
Processing data\train\neutral_bg_outliers
Processing data\train\noisy_bg
Processing data\train\noisy_bg_outliers


Label the images with labelImg

...

Transfer the .xml files to a json file

In [31]:
import xml.etree.ElementTree as ET
import json
import os

def xml_to_json(xml_folder, json_file):
    json_list = []
    for xml_file in os.listdir(xml_folder):
        if xml_file.endswith('.xml'):
            xml_path = os.path.join(xml_folder, xml_file)
            tree = ET.parse(xml_path)
            root = tree.getroot()
            annotations = []
            for member in root.findall('object'):
                annotations.append({
                    "class": member[0].text,
                    "bbox": [
                        int(member[4][0].text),  # xmin
                        int(member[4][1].text),  # ymin
                        int(member[4][2].text),  # xmax
                        int(member[4][3].text)   # ymax
                    ]
                })
            json_list.append({
                "image": root.find('filename').text,
                "annotations": annotations
            })
    
    with open(json_file, 'w') as f:
        json.dump(json_list, f, indent=4)

# 使用函数
xml_to_json(os.path.join('data', 'temp'), 'output_annotations.json')


In [4]:
import json
anno_json = json.load(open(os.path.join('data', 'temp_anno.json')))
anno_json[0]

{'image': 'L1010277.JPG',
 'annotations': [{'class': '5CHF', 'bbox': [1726, 1654, 2486, 2468]},
  {'class': '0.02EUR', 'bbox': [1919, 2978, 2433, 3551]},
  {'class': '0.02EUR', 'bbox': [2569, 2361, 3143, 2918]},
  {'class': '0.05EUR', 'bbox': [3079, 2904, 3759, 3471]},
  {'class': '2EUR', 'bbox': [3069, 1364, 3676, 1994]}]}

In [37]:
from sklearn.model_selection import train_test_split

# 假设data是你的完整数据集
train, val = train_test_split(anno_json, test_size=0.1, random_state=42)  # 剩余85%中再分出15%作为验证集

# 确保分出的数据集的分布是合理的
print(f"Training data: {len(train)}, Validation data: {len(val)}")

Training data: 14, Validation data: 2


In [5]:
import torchvision.models as models

# 加载预训练的 ResNet50 模型
resnet50 = models.resnet50(weights='IMAGENET1K_V2')

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to C:\Users\lenovo/.cache\torch\hub\checkpoints\resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:07<00:00, 12.9MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 